In [1]:
from google.colab import drive


In [2]:
pip install image-classifiers==0.2.2


     |████████████████████████████████| 72 kB 1.4 MB/s 


In [4]:
pip install keras

In [5]:
pip install git+https://github.com/qubvel/classification_models.git


  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-1mdfezul
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-1mdfezul
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 50 kB 2.9 MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-py3-none-any.whl size=20046 sha256=0add090924dc9b37fab82537e6d26173249f2114a36d8c03dc2e057a40c69141
  Stored in directory: /tmp/pip-ephem-wheel-cache-6w2ucfs_/wheels/0b/96/56/27b17c903efc647c51e4f364bfc20aa67f8d3dccad63c4fb4e
Successfully built image-classifiers
  Attempting uninstall: image-classifiers
    Found existing installation: image-classifiers 0.2.2
    Uninstalling image-classifiers-0.2.2:
      Successfully uninstalled image-classifiers-0.2.2


In [ ]:
pip install tensorflow==2.1.0


     |████████████████████████████████| 421.8 MB 22 kB/s 
     |████████████████████████████████| 3.8 MB 38.9 MB/s 
     |████████████████████████████████| 448 kB 61.8 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=7a115bab5d80cdbb5639dbdff0bc9ad9654934f9a85c2ab729fcd8dc991c1d76
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: te

In [6]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from glob import glob
from tqdm import notebook
from sklearn import preprocessing
from keras import Model
from keras.layers import Dropout, Flatten, Dense
from keras import optimizers
import keras
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import sys, os
from sklearn.model_selection import train_test_split
from keras.models import load_model
from keras.models import model_from_json
import h5py

***
Opening Data 
***

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
train = '/content/drive/My Drive/datachallenge/data_train_images.h5'

In [10]:
h5f = h5py.File(train,'r')
train_images = h5f['data_train_images'][:]
h5f.close()


In [11]:
h5f = h5py.File('/content/drive/My Drive/datachallenge/data_train_labels.h5','r')
train_labels = h5f['data_train_labels'][:]
h5f.close()

In [12]:
h5f = h5py.File('/content/drive/My Drive/datachallenge/data_test_images.h5','r')
test_images = h5f['data_test_images'][:]
h5f.close()


***
2D CNN
***

In [13]:
n_classes = 6


In [14]:
# using only the last image
train_1 = train_images[:, 0, :,:].copy()
train_9 = train_images[:, -1, :,:].copy()
train_5 = train_images[:, 5, :,:].copy()

In [15]:
# using only the last image
test_1 = test_images[:, 0, :,:].copy()
test_9 = test_images[:, -1, :,:].copy()
test_5 = test_images[:, 5, :,:].copy()

In [16]:
# remove uncessary dimensions
train_1 = np.squeeze(train_1)
train_9 = np.squeeze(train_9)
train_5 = np.squeeze(train_5)


#set three new channels to pass through network
img2 = np.zeros( (train_1.shape[0], train_1.shape[1],train_1.shape[2], 3 ) )

img2[:,:,:,0] = train_1
img2[:,:,:,1] = train_5
img2[:,:,:,2] = train_9



In [17]:
# remove uncessary dimensions
test_1 = np.squeeze(test_1)
test_9 = np.squeeze(test_9)
test_5 = np.squeeze(test_5)


#set three new channels to pass through network
X_test = np.zeros( (test_1.shape[0], test_1.shape[1],test_1.shape[2], 3 ) )

X_test[:,:,:,0] = test_1
X_test[:,:,:,1] = test_5
X_test[:,:,:,2] = test_9

In [18]:
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')



In [22]:
# build model
base_model = ResNet18(input_shape=(300,200,3), include_top=False)
x = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation='softmax')(x)
model = keras.models.Model(inputs=[base_model.input], outputs=[output])

In [23]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# one hot encoding labels
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_labels.reshape(540,1))
lala =enc.transform(train_labels.reshape(540,1)).toarray()

In [ ]:
# train
model.compile(optimizer='ADAM', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(img2, lala)

15/17 [=========================>....] - ETA: 21s - loss: 1.9295 - accuracy: 0.2833

In [ ]:
model.predict(img2[0:10])

array([[0.31069118, 0.11901982, 0.04378638, 0.18980035, 0.02723224,
        0.3094701 ],
       [0.25729915, 0.08089131, 0.11043912, 0.12252928, 0.09160694,
        0.33723423],
       [0.18785498, 0.05700729, 0.1779961 , 0.10630246, 0.03545982,
        0.4353794 ],
       [0.30436468, 0.05623237, 0.11653003, 0.08759913, 0.0576108 ,
        0.377663  ],
       [0.25337547, 0.09316412, 0.06554199, 0.10797251, 0.04835999,
        0.43158594],
       [0.12961671, 0.14684701, 0.20796438, 0.09396657, 0.02925717,
        0.39234814],
       [0.1755538 , 0.07780797, 0.09107749, 0.07147866, 0.02202848,
        0.56205356],
       [0.22693352, 0.1071989 , 0.09538312, 0.08642829, 0.05691462,
        0.4271415 ],
       [0.1537007 , 0.1031141 , 0.242102  , 0.07339077, 0.03396565,
        0.39372677],
       [0.33190128, 0.07076675, 0.16669321, 0.08148702, 0.04625211,
        0.3028996 ]], dtype=float32)